## Ingest circuits file

In [0]:
dbutils.widgets.text("p_data_date", "")
v_data_date = dbutils.widgets.get("p_data_date")

In [0]:
%run "../conf/configuration_f"

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp

In [0]:
dbutils.secrets.list(scope='fomular1-scope')

In [0]:
formular1_key = dbutils.secrets.get(scope='fomular1-scope', key='formuladl1-key-account')

In [0]:
spark.conf.set("fs.azure.account.key.fomular1dl1.dfs.core.windows.net", formular1_key)

In [0]:
circuits_df = spark.read.option("header", True).option("inferSchema", True).csv(f"abfss://raw@fomular1dl1.dfs.core.windows.net/{v_data_date}/circuits.csv")

In [0]:
circuits_df.show()

In [0]:
circuits_df.printSchema()


In [0]:
circuits_selected_df = circuits_df.select("circuitId", "circuitRef", "name", "location", "country", "lat", "lng", "alt")
circuits_selected_df.show()

In [0]:
circuits_renamed=circuits_selected_df.withColumnRenamed("circuitId", "circuit_id") \
.withColumnRenamed("circuitRef", "circuit_ref") \
.withColumnRenamed("lat", "latitude") \
.withColumnRenamed("lng", "longitude")  \
.withColumnRenamed("alt", "altitude") \
.withColumn("data_date", lit(v_data_date))

circuits_renamed.show()

In [0]:
final_df = circuits_renamed.withColumn("ingestion_date", current_timestamp())
display(final_df)


In [0]:
# final_df.write.parquet("abfss://processed@fomular1dl1.dfs.core.windows.net/circuits", mode="overwrite")

final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.circuits")

In [0]:
spark.read.parquet("abfss://processed@fomular1dl1.dfs.core.windows.net/circuits").show()

In [0]:
%sql 
SELECT * FROM f1_processed.circuits